In [1]:
from agential.cog.agent.reflexion import ReflexionCoTAgent
from agential.cog.prompts.agent.reflexion import (
    REFLEXION_COT_INSTRUCTION_HOTPOTQA,
    HOTPOTQA_FEWSHOT_EXAMPLES_REFLEXION_COT_REFLECT,
    REFLEXION_COT_REFLECT_INSTRUCTION_HOTPOTQA
)
from agential.cog.prompts.benchmark.hotpotqa import (
    HOTPOTQA_FEWSHOT_EXAMPLES_COT
)

from langchain_openai import ChatOpenAI

import dotenv
dotenv.load_dotenv()

import warnings
warnings.filterwarnings('ignore')

llm = ChatOpenAI()

agent = ReflexionCoTAgent(
    llm=llm,
    mode={"qa": "hotpotqa"}
)

In [2]:
question = "VIVA Media AG changed it's name in 2004. What does their new acronym stand for?"
key = "Gesellschaft mit beschränkter Haftung"


agent = ReflexionCoTAgent(
    llm=llm,
    mode={"qa": "hotpotqa"},
)
out = agent.generate(
    question=question, 
    key=key, 
    examples=HOTPOTQA_FEWSHOT_EXAMPLES_COT,
    prompt=REFLEXION_COT_INSTRUCTION_HOTPOTQA, 
    reflection_strategy="reflexion",
    reflect_examples=HOTPOTQA_FEWSHOT_EXAMPLES_REFLEXION_COT_REFLECT,
    reflect_prompt=REFLEXION_COT_REFLECT_INSTRUCTION_HOTPOTQA,
    additional_keys={},
    reflection_additional_keys={},
    max_trials=3,
    patience=3,
    reset=True
)

<PROMPT AGENT=====================================================>
Question: What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?
Thought: Let's think step by step. The eastern sector of Colorado orogeny extends into the High Plains. High Plains rise in elevation from around 1,800 to 7,000 ft, so the answer is 1,800 to 7,000 ft.
Action: Finish[1,800 to 7,000 ft]

Question: Musician and satirist Allie Goertz wrote a song about the "The Simpsons" character Milhouse, who Matt Groening named after who?
Thought: Let's think step by step. Milhouse was named after U.S. president Richard Nixon, so the answer is Richard Nixon.
Action: Finish[Richard Nixon]

Question: Which documentary is about Finnish rock groups, Adam Clayton Powell or The Saimaa Gesture?
Thought: Let's think step by step. Adam Clayton Powell (film) is a documentary about an African-American politician, not Finnish rock groups. So the documentary about Finnish rock groups must

c:\Users\tuvin\anaconda3\envs\agential\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


<OUT AGENT=====================================================>
"Let's think step by step. VIVA Media AG changed its name to VGL Group. So the acronym VGL stands for the new name of the company.\nAction: Finish[VGL Group]"
<OUT AGENT=====================================================>
<PROMPT AGENT=====================================================>
Question: What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?
Thought: Let's think step by step. The eastern sector of Colorado orogeny extends into the High Plains. High Plains rise in elevation from around 1,800 to 7,000 ft, so the answer is 1,800 to 7,000 ft.
Action: Finish[1,800 to 7,000 ft]

Question: Musician and satirist Allie Goertz wrote a song about the "The Simpsons" character Milhouse, who Matt Groening named after who?
Thought: Let's think step by step. Milhouse was named after U.S. president Richard Nixon, so the answer is Richard Nixon.
Action: Finish[Richard Nixon]

Que

In [3]:
out

[{'thought': "Let's think step by step. VIVA Media AG changed its name to VGL Group. So the acronym VGL stands for the new name of the company.",
  'action_type': 'Finish',
  'obs': 'Answer is INCORRECT',
  'answer': 'VGL Group',
  'is_correct': False},
 {'thought': "Let's think step by step. Before answering this question, I need to conduct thorough research on the company's name change event in 2004 to find out the specific new name and acronym used by VIVA Media AG. By doing so, I can ensure that my response is accurate and well-informed. So, the plan is to research the new acronym for VIVA Media AG before providing an answer.",
  'action_type': '',
  'obs': 'Invalid action type, please try again.',
  'answer': 'VGL Group',
  'is_correct': False},
 {'thought': "Let's think step by step. Before answering this question, I need to conduct thorough research on VIVA Media AG's name change event in 2004 to find out what their new acronym stands for. I should look into press releases, offi